# CE-OCL Case Study

In [1]:
import warnings
import os
import sys
sys.path.append(os.path.dirname(os.getcwd())+'//src')
import numpy as np
import pandas as pd
import ce_helpers   
import embed_mip as em

In [2]:
warnings.filterwarnings("ignore")

## Preparation

### Data prep

The (preprocessed) data can be downloaded from: https://www.kaggle.com/datasets/shubamsumbria/statlog-heart-data-set

In [3]:
dataset_name = 'heart'
df = pd.read_csv('../case study/data/statlog.csv')
recode = {"presence": {1: 0, 2:1},
          'cp':{1:'typical angina', 2:'atypical angina', 3:'nonanginal pain', 4:'asymptomatic'},
          'sex':{0:'female', 1:'male'},
          'fbs':{0:'false',1:'true'},
          'restecg':{0:'normal', 1:'having ST-T wave abnormality',2:'left ventricular hypertrophy'},
          'exang':{0:'no', 1:'yes'},
          'slope':{1:'upsloping', 2:'flat', 3:'downsloping'},
          'thal':{3:'normal', 6:'fixed defect', 7:'reversible defect'},
          'ca':{0:'0',1:'1', 2:'2',3:'3'}}
df = df.replace(recode)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,presence
0,70.0,male,asymptomatic,130.0,322.0,false,left ventricular hypertrophy,109.0,no,2.4,flat,3,normal,1
1,67.0,female,nonanginal pain,115.0,564.0,false,left ventricular hypertrophy,160.0,no,1.6,flat,0,reversible defect,0
2,57.0,male,atypical angina,124.0,261.0,false,normal,141.0,no,0.3,upsloping,0,reversible defect,1
3,64.0,male,asymptomatic,128.0,263.0,false,normal,105.0,yes,0.2,flat,1,reversible defect,0
4,74.0,female,atypical angina,120.0,269.0,false,left ventricular hypertrophy,121.0,yes,0.2,upsloping,1,normal,0


In [4]:
df.rename(columns={"cp": "chp", "trestbps": "bp", "chol": "sch", "restecg": "ecg", "thalach": "mhrt", "exang": "exian", "oldpeak": "opk", "ca": "vessel"}, inplace=True)

In [5]:
# outcome variable
target = 'presence'

d = {
    'target': target,
    'numerical':['age','bp','sch', 'mhrt', 'opk']
}

d['categorical'] = df.columns.difference(d['numerical'] + [d['target']])

F_int = [] # integer variables

In [6]:
X = df.drop(target, axis=1)
y = df[target]

X, X_train, X_test, y_train, y_test, F_b, data_pip = ce_helpers.prep_data(X, y, d['numerical'], 
                                                             one_hot_encoding = True, scaling = True)


In [7]:
X_train.head()

,chp_asymptomatic,chp_atypical angina,chp_nonanginal pain,chp_typical angina,ecg_having ST-T wave abnormality,ecg_left ventricular hypertrophy,ecg_normal,exian_no,exian_yes,fbs_false,...,thal_reversible defect,vessel_0,vessel_1,vessel_2,vessel_3,age,bp,sch,mhrt,opk
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.583333,0.571429,0.095890,0.786260,0.258065
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,0.625000,0.418367,0.246575,0.687023,0.080645
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.770833,0.183673,0.196347,0.465649,0.016129
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,0.458333,0.469388,0.392694,0.389313,0.677419
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.270833,0.265306,0.260274,0.938931,0.129032


### Train predictive models

In [8]:
version = dataset_name+'_v1'
alg_list_cf = ['mlp']
outcome_dict = {'counterfactual_heart':{'task': 'binary', 'X features': X_train.columns, 
                                        'class': target, 'alg_list': alg_list_cf,
                                        'X_train':X_train, 'X_test':X_test,
                                        'y_train':y_train, 'y_test':y_test}}

## uncomment if models should be trained
ce_helpers.train_models(outcome_dict, version)
performance = ce_helpers.perf_trained_models(version, outcome_dict)
performance

Learning a constraint for counterfactual_heart
Training mlp
------------- Initialize grid  ----------------
------------- Running model  ----------------
Algorithm = mlp, metric = None
saving... results/mlp_counterfactual_heart_trained.pkl
------------- Model evaluation  ----------------
-------------------training evaluation-----------------------
Train Score: 1.0
-------------------testing evaluation-----------------------
Test Score: 0.8208333333333333
------------- Save results  ----------------

Saving the performance...
Done!


,save_path,seed,cv_folds,task,parameters,best_params,valid_score,train_score,test_score,outcome,alg
0,results/mlp/heart_v1_counterfactual_heart_mode...,0,5,binary,"{'hidden_layer_sizes': [(10,), (20,), (50,), (...","{'hidden_layer_sizes': (50,)}",0.886579,1.0,0.820833,counterfactual_heart,mlp


In [9]:
# load models
algorithms = {'counterfactual_heart':'mlp'}
y_pred, y_pred_0, X_test_0, models = ce_helpers.load_model(algorithms, outcome_dict, 'counterfactual_heart')  # it should be X_test instead of X

clf = models['counterfactual_heart']

In [10]:
## for coherence
F_r = d['numerical']
categorical = df.columns.difference(d['numerical'] + [target])

F_coh = {}
for f in categorical:
    F_coh[f] = [i for i in list(X_train.columns.difference(d['numerical'] + [target])) if i.startswith('%s_' % f)]

### Prep for Optimization

Define how the counterfactual constraint should look like.

In [11]:
constraints_embed = ['counterfactual_heart']
objectives_embed = {}

In [12]:
model_master = em.model_selection(performance[performance['alg']==algorithms['counterfactual_heart']], constraints_embed, objectives_embed)
model_master['lb'] = 0.5  # this can be changed but it is generally equal to 0.5
model_master['ub'] = None
model_master['SCM_counterfactuals'] = None
model_master['features'] = [[col for col in X.columns]]
model_master

                outcome model_type  \
0  counterfactual_heart        mlp   

                                           save_path    task  objective  
0  results/mlp/heart_v1_counterfactual_heart_mode...  binary          0  


,outcome,model_type,save_path,task,objective,lb,ub,SCM_counterfactuals,features
0,counterfactual_heart,mlp,results/mlp/heart_v1_counterfactual_heart_mode...,binary,0,0.5,None,None,"[chp_asymptomatic, chp_atypical angina, chp_no..."


X1: points in X that have 1 as label. They will be used as trust region

In [13]:
y_ix_1 = np.where(y==1)
X1 = X.iloc[y_ix_1[0],:].copy().reset_index(drop=True, inplace=False)

### Factual instance

In [14]:
# My sample
u_index = 3
u = X_test_0.iloc[u_index,:]
print(u)
print('predicted label: %d' % (clf.predict([u])))

chp_asymptomatic                    0.000000
chp_atypical angina                 1.000000
chp_nonanginal pain                 0.000000
chp_typical angina                  0.000000
ecg_having ST-T wave abnormality    0.000000
ecg_left ventricular hypertrophy    0.000000
ecg_normal                          1.000000
exian_no                            1.000000
exian_yes                           0.000000
fbs_false                           1.000000
fbs_true                            0.000000
sex_female                          0.000000
sex_male                            1.000000
slope_downsloping                   0.000000
slope_flat                          0.000000
slope_upsloping                     1.000000
thal_fixed defect                   0.000000
thal_normal                         1.000000
thal_reversible defect              0.000000
vessel_0                            1.000000
vessel_1                            0.000000
vessel_2                            0.000000
vessel_3  

## Part A: validity, proximity

In [15]:
sp = False
mu = 0
tr_region = False
enlarge_tr = False
num_counterfactuals = 1

In [16]:
# features that can only increase (become larger)
L = []

# immutable features
I = []


Pers_I = [] # variables that must be considered for person specific immutable features

P = []

### Optimization

In [17]:
CEs, CEs_, final_model = ce_helpers.opt(X, X1, u, F_r, F_b, F_int, F_coh, I, L, Pers_I, P, 
                                        sp, mu, tr_region, enlarge_tr, num_counterfactuals, model_master, data_pip)

Embedding constraints for counterfactual_heart
['chp_asymptomatic', 'chp_atypical angina', 'chp_nonanginal pain', 'chp_typical angina', 'ecg_having ST-T wave abnormality', 'ecg_left ventricular hypertrophy', 'ecg_normal', 'exian_no', 'exian_yes', 'fbs_false', 'fbs_true', 'sex_female', 'sex_male', 'slope_downsloping', 'slope_flat', 'slope_upsloping', 'thal_fixed defect', 'thal_normal', 'thal_reversible defect', 'vessel_0', 'vessel_1', 'vessel_2', 'vessel_3', 'age', 'bp', 'sch', 'mhrt', 'opk']
Academic license - for non-commercial use only - expires 2023-07-04
Using license file C:\gurobi912\win64\gurobi.lic
Changed value of parameter PoolSolutions to 101
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolSearchMode to 1
   Prev: 0  Min: 0  Max: 2  Default: 0
OBJ: 0.051319272107715475
The optimal solution is: [-0.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 0.0, 1.0, 0.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, 0.41298249340692356, 0

### Visualize DF

In [18]:
df_1 = ce_helpers.visualise_changes(clf, d, F_coh=F_coh, method = 'CE-OCL', CEs=CEs, CEs_ = CEs_, only_changes=True)
df_1

,age,bp,sch,mhrt,opk,presence,chp,ecg,exian,fbs,sex,slope,thal,vessel
original,49.00,130.00,265.98,171.01,0.60,0.0,atypical angina,normal,no,false,male,upsloping,normal,0
sol0,48.82,139.28,328.09,153.98,1.05,1.0,-,-,-,-,-,-,-,-


### Evaluation

In [19]:
df_orig = ce_helpers.visualise_changes(clf, d, F_coh=F_coh, method = 'CE-OCL', CEs=CEs, CEs_ = CEs_, only_changes=True)
CE_perf = ce_helpers.evaluation(df_orig, d).set_index(pd.Index(['Part A']))
CE_perf

,validity,cat_prox,cont_prox,sparsity,cat_diver,cont_diver,cont_count_divers
Part A,1.0,0.0,-89.05,0.0,None,None,None


---

## Part B: validity, proximity, sparsity

In [20]:
sp = True
mu = 10000
tr_region = False
num_counterfactuals = 1

In [21]:
# features that can only increase (become larger)
L = []

# immutable features
I = []

# conditionally mutable features
Pers_I = []

P = []

### optimization

In [22]:
CEs, CEs_, final_model = ce_helpers.opt(X, X1, u, F_r, F_b, F_int, F_coh, I, L, Pers_I, P, sp, mu, 
                           tr_region, enlarge_tr, num_counterfactuals, model_master, data_pip)

Embedding constraints for counterfactual_heart
['chp_asymptomatic', 'chp_atypical angina', 'chp_nonanginal pain', 'chp_typical angina', 'ecg_having ST-T wave abnormality', 'ecg_left ventricular hypertrophy', 'ecg_normal', 'exian_no', 'exian_yes', 'fbs_false', 'fbs_true', 'sex_female', 'sex_male', 'slope_downsloping', 'slope_flat', 'slope_upsloping', 'thal_fixed defect', 'thal_normal', 'thal_reversible defect', 'vessel_0', 'vessel_1', 'vessel_2', 'vessel_3', 'age', 'bp', 'sch', 'mhrt', 'opk']
Changed value of parameter PoolSolutions to 101
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolSearchMode to 1
   Prev: 0  Min: 0  Max: 2  Default: 0
OBJ: 10000.103969578797
The optimal solution is: [-0.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 0.0, 1.0, 0.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, 0.41666666666666663, 0.36734693877551017, 0.642077843605986, 0.763358778625954, 0.0967741935483871]


### Visualize DF

In [23]:
df_2 = ce_helpers.visualise_changes(clf, d, F_coh=F_coh, method = 'CE-OCL', CEs=CEs, CEs_ = CEs_, only_changes=True)
df_2

,age,bp,sch,mhrt,opk,presence,chp,ecg,exian,fbs,sex,slope,thal,vessel
original,49.0,130.0,265.98,171.01,0.6,0.0,atypical angina,normal,no,false,male,upsloping,normal,0
sol0,-,-,407.24,-,-,1.0,-,-,-,-,-,-,-,-


### Evaluation

In [24]:
df_orig = ce_helpers.visualise_changes(clf, d, F_coh=F_coh, method='CE-OCL', CEs=CEs, CEs_=CEs_)
CE_perf = pd.concat([CE_perf, ce_helpers.evaluation(df_orig, d)]).set_index(pd.Index(['Part A', 'Part B']))
CE_perf

,validity,cat_prox,cont_prox,sparsity,cat_diver,cont_diver,cont_count_divers
Part A,1.0,0.0,-89.05,0.00,None,None,None
Part B,1.0,1.0,-141.26,0.92,None,None,None


---

## Part C: validity, proximity, sparsity, diversity

In [25]:
sp = True
tr_region = False
num_counterfactuals = 3

In [26]:
# features that can only increase (become larger)
L = []

# immutable features
I = []

# conditionally mutable features
Pers_I = []

P = []

### Optimization

In [27]:
CEs, CEs_, final_model = ce_helpers.opt(X, X1, u, F_r, F_b, F_int,F_coh, I, L, Pers_I, P, sp, mu, 
                           tr_region, enlarge_tr, num_counterfactuals, model_master, data_pip)

Embedding constraints for counterfactual_heart
['chp_asymptomatic', 'chp_atypical angina', 'chp_nonanginal pain', 'chp_typical angina', 'ecg_having ST-T wave abnormality', 'ecg_left ventricular hypertrophy', 'ecg_normal', 'exian_no', 'exian_yes', 'fbs_false', 'fbs_true', 'sex_female', 'sex_male', 'slope_downsloping', 'slope_flat', 'slope_upsloping', 'thal_fixed defect', 'thal_normal', 'thal_reversible defect', 'vessel_0', 'vessel_1', 'vessel_2', 'vessel_3', 'age', 'bp', 'sch', 'mhrt', 'opk']
Changed value of parameter PoolSolutions to 103
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolSearchMode to 1
   Prev: 0  Min: 0  Max: 2  Default: 0
OBJ: 10000.103969578797
The optimal solution is: [-0.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 0.0, 1.0, 0.0, -0.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, 0.41666666666666663, 0.36734693877551017, 0.642077843605986, 0.763358778625954, 0.0967741935483871]


### Visualise DF

In [28]:
df_3 = ce_helpers.visualise_changes(clf, d, F_coh=F_coh, method = 'CE-OCL', CEs=CEs, CEs_ = CEs_, only_changes=True)
df_3

,age,bp,sch,mhrt,opk,presence,chp,ecg,exian,fbs,sex,slope,thal,vessel
original,49.0,130.0,265.98,171.01,0.6,0.0,atypical angina,normal,no,false,male,upsloping,normal,0
sol0,-,-,407.24,-,-,1.0,-,-,-,-,-,-,-,-
sol1,-,-,393.92,175.14,-,1.0,-,-,-,-,-,-,-,-
sol2,-,-,404.04,-,0.47,1.0,-,-,-,-,-,-,-,-


### Evaluation

In [29]:
df_orig = ce_helpers.visualise_changes(clf, d, F_coh=F_coh, method='CE-OCL', CEs=CEs, CEs_=CEs_)
CE_perf = pd.concat([CE_perf, ce_helpers.evaluation(df_orig, d)]).set_index(pd.Index(['Part A', 'Part B', 'Part C']))
CE_perf

,validity,cat_prox,cont_prox,sparsity,cat_diver,cont_diver,cont_count_divers
Part A,1.0,0.0,-89.05,0.00,NaN,NaN,NaN
Part B,1.0,1.0,-141.26,0.92,NaN,NaN,NaN
Part C,1.0,1.0,-137.17,0.87,0.0,11.72,0.18


---

## Part D: validity, proximity, sparsity, diversity, actionability

In [30]:
sp = True
tr_region = False
num_counterfactuals = 3

In [31]:
# features that can only increase (become larger)
L = []

# immutable features
I = ['sex_male','sex_female', 'age']


Pers_I = [] # variables that must be considered for person specific immutable features

P = ['bp','sch', 'mhrt', 'opk']

### Optimization

In [32]:
CEs, CEs_, final_model = ce_helpers.opt(X, X1, u, F_r, F_b, F_int, F_coh, I, L, Pers_I, P, sp, mu, 
                           tr_region, enlarge_tr, num_counterfactuals, model_master, data_pip)

Embedding constraints for counterfactual_heart
['chp_asymptomatic', 'chp_atypical angina', 'chp_nonanginal pain', 'chp_typical angina', 'ecg_having ST-T wave abnormality', 'ecg_left ventricular hypertrophy', 'ecg_normal', 'exian_no', 'exian_yes', 'fbs_false', 'fbs_true', 'sex_female', 'sex_male', 'slope_downsloping', 'slope_flat', 'slope_upsloping', 'thal_fixed defect', 'thal_normal', 'thal_reversible defect', 'vessel_0', 'vessel_1', 'vessel_2', 'vessel_3', 'age', 'bp', 'sch', 'mhrt', 'opk']
Changed value of parameter PoolSolutions to 103
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolSearchMode to 1
   Prev: 0  Min: 0  Max: 2  Default: 0
OBJ: 10000.103969578799
The optimal solution is: [-0.0, 1.0, -0.0, -0.0, -0.0, -0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, -0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, 0.41666666666666663, 0.3673469387755102, 0.6420778436084014, 0.7633587786259539, 0.0967741935483871]


### Visualize DF

In [33]:
df_4 = ce_helpers.visualise_changes(clf, d, F_coh=F_coh, method = 'CE-OCL', CEs=CEs, CEs_ = CEs_, only_changes=True)
df_4

,age,bp,sch,mhrt,opk,presence,chp,ecg,exian,fbs,sex,slope,thal,vessel
original,49.0,130.0,265.98,171.01,0.6,0.0,atypical angina,normal,no,false,male,upsloping,normal,0
sol0,-,-,407.24,-,-,1.0,-,-,-,-,-,-,-,-
sol1,-,-,393.92,175.14,-,1.0,-,-,-,-,-,-,-,-
sol2,-,-,-,124.37,-,1.0,-,-,-,-,-,-,-,-


### Evaluation

In [34]:
df_orig = ce_helpers.visualise_changes(clf, d, F_coh=F_coh, method='CE-OCL', CEs=CEs, CEs_=CEs_)
CE_perf = pd.concat([CE_perf, ce_helpers.evaluation(df_orig, d)]).set_index(pd.Index(['Part A', 'Part B',
                                                                                            'Part C', 'Part D']))
CE_perf

,validity,cat_prox,cont_prox,sparsity,cat_diver,cont_diver,cont_count_divers
Part A,1.0,0.0,-89.05,0.00,NaN,NaN,NaN
Part B,1.0,1.0,-141.26,0.92,NaN,NaN,NaN
Part C,1.0,1.0,-137.17,0.87,0.0,11.72,0.18
Part D,1.0,1.0,-106.66,0.90,0.0,128.02,0.15


---

## Part E: validity, proximity, sparsity, diversity, actionability, trust region

In [35]:
sp = True
tr_region = True
enlarge_tr = False
num_counterfactuals = 3

In [36]:
# features that can only increase (become larger)
L = []

# immutable features
I = ['sex_male','sex_female', 'age']


Pers_I = [] # variables that must be considered for person specific immutable features

P = ['bp','sch', 'mhrt', 'opk']

### Optimization

In [37]:
CEs, CEs_, final_model = ce_helpers.opt(X, X1, u, F_r, F_b, F_int, F_coh, I, L, Pers_I, P, sp, mu, 
                           tr_region, enlarge_tr, num_counterfactuals, model_master, data_pip)

Generating constraints for the trust region using 120 samples.
The trust region is not being enlarged.
... Trust region defined.
Embedding constraints for counterfactual_heart
['chp_asymptomatic', 'chp_atypical angina', 'chp_nonanginal pain', 'chp_typical angina', 'ecg_having ST-T wave abnormality', 'ecg_left ventricular hypertrophy', 'ecg_normal', 'exian_no', 'exian_yes', 'fbs_false', 'fbs_true', 'sex_female', 'sex_male', 'slope_downsloping', 'slope_flat', 'slope_upsloping', 'thal_fixed defect', 'thal_normal', 'thal_reversible defect', 'vessel_0', 'vessel_1', 'vessel_2', 'vessel_3', 'age', 'bp', 'sch', 'mhrt', 'opk']
Changed value of parameter PoolSolutions to 103
   Prev: 10  Min: 1  Max: 2000000000  Default: 10
Changed value of parameter PoolSearchMode to 1
   Prev: 0  Min: 0  Max: 2  Default: 0
OBJ: 60002.06426384043
The optimal solution is: [-0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, -0.0, 1.0, -0.0, 1.0, -0.0, 1.0, -0.0, -0.0, -0.0, 0.41666666666666663

#### Visualize DF

In [38]:
df_5 = ce_helpers.visualise_changes(clf, d, F_coh=F_coh, method = 'CE-OCL', CEs=CEs, CEs_ = CEs_, only_changes=True)
df_5

,age,bp,sch,mhrt,opk,presence,chp,ecg,exian,fbs,sex,slope,thal,vessel
original,49.0,130.00,265.98,171.01,0.60,0.0,atypical angina,normal,no,false,male,upsloping,normal,0
sol0,-,111.77,253.81,152.70,0.00,1.0,nonanginal pain,-,-,-,-,-,-,-
sol1,-,137.00,258.50,147.01,1.55,1.0,asymptomatic,left ventricular hypertrophy,-,-,-,flat,reversible defect,-
sol2,-,140.61,274.70,128.61,0.49,1.0,asymptomatic,left ventricular hypertrophy,yes,-,-,-,reversible defect,-


### Evaluating

In [39]:
df_orig = ce_helpers.visualise_changes(clf, d, F_coh=F_coh, method='CE-OCL', CEs=CEs, CEs_=CEs_)
CE_perf = pd.concat([CE_perf, ce_helpers.evaluation(df_orig, d)]).set_index(pd.Index(['Part A', 'Part B',
                                                                                            'Part C', 'Part D',
                                                                                            'Part E']))
CE_perf

,validity,cat_prox,cont_prox,sparsity,cat_diver,cont_diver,cont_count_divers
Part A,1.0,0.00,-89.05,0.00,NaN,NaN,NaN
Part B,1.0,1.00,-141.26,0.92,NaN,NaN,NaN
Part C,1.0,1.00,-137.17,0.87,0.00,11.72,0.18
Part D,1.0,1.00,-106.66,0.90,0.00,128.02,0.15
Part E,1.0,0.62,-50.19,0.46,0.42,50.25,0.56


---

## Table

In [40]:
df_complete = pd.concat([df_1, df_2, df_3, df_4, df_5])
df_complete

,age,bp,sch,mhrt,opk,presence,chp,ecg,exian,fbs,sex,slope,thal,vessel
original,49.0,130.0,265.98,171.01,0.6,0.0,atypical angina,normal,no,false,male,upsloping,normal,0
sol0,48.82,139.28,328.09,153.98,1.05,1.0,-,-,-,-,-,-,-,-
original,49.0,130.0,265.98,171.01,0.6,0.0,atypical angina,normal,no,false,male,upsloping,normal,0
sol0,-,-,407.24,-,-,1.0,-,-,-,-,-,-,-,-
original,49.0,130.0,265.98,171.01,0.6,0.0,atypical angina,normal,no,false,male,upsloping,normal,0
sol0,-,-,407.24,-,-,1.0,-,-,-,-,-,-,-,-
sol1,-,-,393.92,175.14,-,1.0,-,-,-,-,-,-,-,-
sol2,-,-,404.04,-,0.47,1.0,-,-,-,-,-,-,-,-
original,49.0,130.0,265.98,171.01,0.6,0.0,atypical angina,normal,no,false,male,upsloping,normal,0
sol0,-,-,407.24,-,-,1.0,-,-,-,-,-,-,-,-
